In [ ]:
# Bag of words
# Система позволяет искать тексты из ограниченного количества документов, тесно связаны с заданной темой. 
# Чтобы использовать возможности NLP, объединим методологию поиска с методами оценки семантического сходства.

In [4]:
import csv
import pandas as pd

data = 'D:/Python_Projects/Recomendation_Systems/kinopoisk-top250.csv'
df = pd.read_csv(data)

df = df[~df.isna()]

df = df.iloc[:5]
df.head()

,rating,movie,rating_ball,overview,director,screenwriter
0,0,Побег из Шоушенка,9.111,Бухгалтер Энди Дюфрейн обвинён в убийстве собс...,Фрэнк Дарабонт,Фрэнк Дарабонт; Стивен Кинг
1,1,Зеленая миля,9.062,Пол Эджкомб — начальник блока смертников в тюр...,Фрэнк Дарабонт,Фрэнк Дарабонт; Стивен Кинг
2,2,Форрест Гамп,8.913,От лица главного героя Форреста Гампа; слабоум...,Роберт Земекис,Эрик Рот; Уинстон Грум
3,3,Список Шиндлера,8.817,Фильм рассказывает реальную историю загадочног...,Стивен Спилберг,Стивен Зеллиан; Томас Кенилли
4,4,1+1,8.807,Пострадав в результате несчастного случая; бог...,Оливье Накаш; Эрик Толедано,Оливье Накаш; Филипп Поццо ди Борго; Эрик Т...


In [13]:
#Удаляем слишком короткие и слишком длинные предложения из набора данных с помощью MIN_WORDS и MAX_WORDS. 
#Удаляем стоп слова, которые не помогают извлечь специфику предложения.

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

STOPWORDS = set(stopwords.words('english'))
MIN_WORDS = 5
MAX_WORDS = 100

 #Очистка.Перевод строк в нижний регистр, удаление символов не являющихся словами (знаки препинания, кавычки и т.д.)
    
PATTERN_S = re.compile("\'s")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 

def clean_text(text):
  
    text = str(text).lower()  # lowercase text
    # replace the matched string with ' '
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

#Лемматизация, токенизация, обрезка и удаление стоп слов.
 
def tokenizer(overview, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True):
  
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(overview)]
    else:
        tokens = [w for w in word_tokenize(overview)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens

#Уадление неверных символов (в новой колонке clean_sentence)
#Лемматизация, токенизация слов в список слов (в новой колонке tok_lem_sentence )

def clean_overview(df):   
    
    print('Cleaning overview')
    df['clean_overview'] = df['overview'].apply(clean_text)
    df['tok_lem_overview'] = df['clean_overview'].apply(
        lambda x: tokenizer(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS))
    return df
    
df = clean_overview(df)

Cleaning overview


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sneda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sneda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sneda\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
#Функция, которая будет ранжировать лучшие рекомендации с учетом косинусного расстояния между векторами

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

#Использует сумму косинусного расстояния для всех токенов и возвращает наилучшее совпадение
def extract_best_indices(m, topk, mask=None):
 
    # Возвращает сумму по всем токенам косинуса для каждого предложения
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # от самого высокого idx до наименьшего балла
    if mask is not None:
        assert mask.shape == m.shape
        mask = mask[index]
    else:
        mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #исключает косинусное расстояние 0
    best_index = index[mask][:topk]  
    return best_index

In [25]:
# TF-IDF
# После расчета TF-IDF (веса слов с учетом их частоты в тексте, нормированной на частоту во всем корпусе документов), 
# сгенерируем вектор встраивания для каждого описания фильма. Эти функции хранятся в матрице функций tfidf_mat, 
# где каждая строка это запись описания фильма, встроенная в вектор функций. 
# Когда получим запрос из пользовательского ввода, встроим его в одно и то же векторное пространство и сравним
# один за другим функцию предложения запроса embed_query с векторами предложений матрицы внедрения tfidf_mat.

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_recommendations_tfidf(overview, tfidf_mat):
    
      # Вставить запрос
    tokens_query = [str(tok) for tok in tokenizer(overview)]
    embed_query = vectorizer.transform(tokens_query)
    
    # Создать список схожести между запросом и датасетом
    mat = cosine_similarity(embed_query, tfidf_mat)
    
    # Лучшее косинусное расстояния для каждого токена независимо
    best_index = extract_best_indices(mat, topk=3)
    return best_index

# адаптация стоп слов
token_stop = tokenizer(' '.join(stopwords.words('english')), lemmatize=False)

# Fit TFIDF
vectorizer = TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer) 
tfidf_mat = vectorizer.fit_transform(df['overview'].values)

# Возврат лучших трех совпадений между запросом и датасетом
test_overview = 'От лица главного героя Форреста Гампа' 
best_index = get_recommendations_tfidf(test_overview, tfidf_mat)

display(df[['movie', 'rating_ball', 'overview']].iloc[best_index])

C:\Users\sneda\anaconda3\envs\PythonGPU\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\sneda\anaconda3\envs\PythonGPU\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  warnings.warn(


,movie,rating_ball,overview
2,Форрест Гамп,8.913,От лица главного героя Форреста Гампа; слабоум...
4,1+1,8.807,Пострадав в результате несчастного случая; бог...
3,Список Шиндлера,8.817,Фильм рассказывает реальную историю загадочног...
